In [ ]:
!pip uninstall openai
!pip install openai
!pip install scikit-learn matplotlib seaborn
!pip install --upgrade openai
!pip install ace_tools

Found existing installation: openai 1.57.4
Uninstalling openai-1.57.4:
  Would remove:
    /usr/local/bin/openai
    /usr/local/lib/python3.10/dist-packages/openai-1.57.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/openai/*
Proceed (Y/n)? y
  Successfully uninstalled openai-1.57.4
  Using cached openai-1.57.4-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.57.4-py3-none-any.whl (390 kB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import openai
from openai import OpenAI
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('openai_key_')
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [ ]:
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from datetime import datetime

EXPERIMENT-1

In [ ]:
# @title
import os
import random

# Function to read a document and split it into words
def read_document_and_split(file_path):
    with open(file_path, 'r') as file:
        text = file.read().strip()
    words = text.split()  # Split the text into words
    return words

# Function to shuffle words in files from all folders
def prepare_shuffled_words_for_all_files():
    # Prompt the user to input the base path containing folders A, B, C
    base_path = input("Enter the path to the base folder containing subfolders (e.g., A, B, C): ")

    if not os.path.exists(base_path):
        print("Invalid base path. Please check and try again.")
        return

    # Output folder for processed files
    output_folder = os.path.join(base_path, "Processed_Files")
    os.makedirs(output_folder, exist_ok=True)  # Create the output directory

    for folder_name in ['A', 'B', 'C']:
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.exists(folder_path):
            print(f"Folder '{folder_name}' not found. Skipping.")
            continue

        # Create an output subfolder
        output_subfolder = os.path.join(output_folder, folder_name)
        os.makedirs(output_subfolder, exist_ok=True)

        # Iterate through each text file in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".txt"):
                file_path = os.path.join(folder_path, file_name)
                print(f"\nProcessing file: {file_name}")

                # Read and shuffle words
                original_words = read_document_and_split(file_path)
                shuffled_words = original_words.copy()
                random.shuffle(shuffled_words)

                # Print shuffled words for the current file
                print(f"Shuffled Words for {file_name}: {shuffled_words}\n")

                # Save shuffled and original words to a new file
                output_file_path = os.path.join(output_subfolder, f"processed_{file_name}")
                with open(output_file_path, 'w') as output_file:
                    output_file.write(f"Shuffled Words:\n{' '.join(shuffled_words)}\n\n")
                    output_file.write(f"Original Words:\n{' '.join(original_words)}")

                print(f"Processed file saved as: {output_file_path}")

# Run the function
prepare_shuffled_words_for_all_files()


Enter the path to the base folder containing subfolders (e.g., A, B, C): /content/drive/MyDrive/Data (2) (1)/Data

Processing file: A-30.txt
Shuffled Words for A-30.txt: ['down', 'pulled', 'hooks,', 'functionality.', 'last', 'I', 'I', 'mixing,', 'right', 'any', 'des', 'dating', 'are', 'no', '“mixité”', 'public', 'and', 'true', 'before', 'precious:', 'But', 'the', 'and', 'men', 'what’s', 'to', 'hang', 'opens', 'city’s', 'even', 'women.', 'their', 'team', 'in.', 'didn’t', 'is:', 'pools', 'see', 'that', 'on', 'to', 'the', 'breathe', 'swimmers', 'it', 'stuff', 'a', 'showers', 'the', 'for', 'kid.', 'two', 'year', 'and', 'spilling', 'the', 'boy', 'the', 'your', 'considers', 'in', 'a', 'out.', 'until', 'lanes,', 'her', 'architectural', 'rising', 'even', 'I’ve', 'feels', 'Many', 'room,', 'to', 'the', 'guide,', 'notably', 'quickly', 'I', 'in', 'skylight', 'swimming', 'their', 'coed', '1930', 'on', 'and', 'quickly', 'the', 'and', 'with', 'solved', 'French', 'You', 'they', 'I', 'close', 'in', 'th

In [116]:
# @title
import os
import random
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import openai

# Function for getting completion from the specified model
def get_completion(prompt, model="gpt-4o", seed=420):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        seed=seed,
    )
    return response.choices[0].message.content




# Function for reconstruction using GPT models
def llm_reconstruction(shuffled_chunks, p, model):
    prompt = f"""
You are an advanced language model.Reconstruct exactly {p} sentences from the following shuffled groups of words.
- Use all words in a group exactly once to form one complete and grammatically correct sentence.
- Do not use words from one group in another.
- Maintain original punctuation attached to words.

Label each reconstructed sentence as:
Sentence 1: <reconstructed sentence>
Sentence 2: <reconstructed sentence>
...
Sentence {p}: <reconstructed sentence>

Input: {shuffled_chunks}
    """
    try:
        reconstructed_paragraph = get_completion(prompt, model=model)
        if not reconstructed_paragraph:
            raise ValueError("No response from the model.")

        # Extract sentences
        sentences = re.findall(r'^Sentence \d+:(.*)', reconstructed_paragraph, re.MULTILINE)
        reconstructed_sentences = {f"Sentence {i+1}": sentence.strip() for i, sentence in enumerate(sentences)}
        return reconstructed_sentences, len(sentences)
    except Exception as e:
        print(f"Error during LLM reconstruction: {e}")
        print(f"shuffled_chunks: {shuffled_chunks}")
        return {}, 0

# Function to prepare shuffled sentences with delimiters
def prepare_shuffled_sentences(file_path):
    with open(file_path, 'r') as file:
        text = file.read().strip()

    sentences = re.split(r'(?<=[.!?]) +', text)
    shuffled_sentences = []

    for sentence in sentences:
        words = sentence.split()
        random.shuffle(words)
        shuffled_sentence = ' '.join(words)
        shuffled_sentences.append(shuffled_sentence)

    shuffled_text = ' * '.join(shuffled_sentences) + ' *'
    return shuffled_text, sentences

# Function to compute embeddings
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Function to process a specific folder and compute results
def process_folder(base_path, folder_name):
    results = []
    folder_path = os.path.join(base_path, folder_name)

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            try:
                p = int(file_name.split("-")[1].split(".")[0])
            except (IndexError, ValueError):
                print(f"Could not extract `p` value from file name {file_name}. Skipping.")
                continue

            with open(file_path, 'r') as file:
                content = file.read()

            # Prepare shuffled text
            shuffled_text, _ = prepare_shuffled_sentences(file_path)
            print(f"Processing File {file_name}:")
            #print(f"Shuffled Text:\n{shuffled_text}")

            # Attempt Reconstruction
            reconstructed_paragraph, num_sentences = llm_reconstruction(shuffled_text, p, model="gpt-4o")
            print(f"\nReconstructed Paragraph for {file_name}:\n{reconstructed_paragraph}")
            print(f"Number of Sentences Reconstructed: {num_sentences}")

            if not reconstructed_paragraph:
                print(f"Reconstruction failed for {file_name}. Skipping similarity computation.")
                continue

            # Compute cosine similarity
            try:
                embed_original = get_embedding(content, model="text-embedding-3-large")
                embed_rebuilt = get_embedding(' '.join(reconstructed_paragraph.values()), model="text-embedding-3-large")
                similarity = cosine_similarity([embed_original], [embed_rebuilt])[0][0]

                # Append results
                results.append({
                    "File": file_name,
                    "p": p,
                    "Cosine Similarity": similarity,
                    "Reconstructed Sentences": num_sentences,
                    "Dataset": folder_name,
                })
            except Exception as embed_error:
                print(f"Error during embedding computation for {file_name}: {str(embed_error)}")

    # Calculate average and standard deviation for similarity
    df = pd.DataFrame(results)
    if not df.empty:
        avg_similarity = df["Cosine Similarity"].mean()
        std_similarity = df["Cosine Similarity"].std()
        print(f"Dataset: {folder_name} - Average Similarity: {avg_similarity}, Standard Deviation: {std_similarity}")
    else:
        avg_similarity, std_similarity = 0, 0
        print(f"Dataset: {folder_name} - No valid results.")

    return df, avg_similarity, std_similarity

# Function to calculate and create a summary table across datasets
def create_summary_table(results_by_folder):
    summary_data = {
        "Dataset": [],
        "(Avg, Std)": []
    }

    for folder_name, (df, avg_similarity, std_similarity) in results_by_folder.items():
        summary_data["Dataset"].append(folder_name)
        summary_data["(Avg, Std)"].append(f"({avg_similarity:.2f}, {std_similarity:.2f})")

    summary_df = pd.DataFrame(summary_data)
    return summary_df

# Define the base path to the dataset
base_path = "/content/drive/MyDrive/Data (2) (1)/Data/Processed_Files1"

# Process each folder individually and store results
results_by_folder = {}
print("Processing folder A:")
results_by_folder["A"] = process_folder(base_path, "A")

print("\nProcessing folder B:")
results_by_folder["B"] = process_folder(base_path, "B")

print("\nProcessing folder C:")
results_by_folder["C"] = process_folder(base_path, "C")

# Generate the summary table
print("\nGenerating summary table...")
summary_table = create_summary_table(results_by_folder)

# Display the summary table
print("\nSummary Table:")
print(summary_table)


Processing folder A:
Processing File processed_A-30.txt:

Reconstructed Paragraph for processed_A-30.txt:
{'Sentence 1': 'In college, I competed on a high school swimming team.', 'Sentence 2': 'You quickly count the views from the long, changing windows.', 'Sentence 3': 'The historical guide, which considers the Tower of Amiraux, reflects changing attitudes.', 'Sentence 4': 'The pools in Paris are coed, and the rules stress historical views.', 'Sentence 5': 'The guide considers the historical side of the pools in Paris.', 'Sentence 6': 'The changing rooms in the pool are lined with historical tiles.', 'Sentence 7': 'The pool in Paris is coed, and the rules stress historical views.', 'Sentence 8': 'The guide considers the historical side of the pools in Paris.', 'Sentence 9': 'The changing rooms in the pool are lined with historical tiles.', 'Sentence 10': 'The pool in Paris is coed, and the rules stress historical views.', 'Sentence 11': 'The guide considers the historical side of the 

EXPERIMENT-2

In [ ]:
# @title
import os
import random
import re

# Common abbreviations to exclude from sentence splitting
ABBREVIATIONS = {"Mr.", "Mrs.", "Dr.", "Ms.", "Jr.", "Sr.", "St.", "Prof.", "Inc.", "Ltd.", "Co."}

def preprocess_text(text):
    """Preprocesses the text to protect abbreviations from being split."""
    for abbrev in ABBREVIATIONS:
        # Replace abbreviations with a placeholder to prevent splitting
        text = text.replace(abbrev, abbrev.replace(".", "<DOT>"))
    return text

def postprocess_sentences(sentences):
    """Reverts placeholders in sentences back to their original abbreviations."""
    return [sentence.replace("<DOT>", ".") for sentence in sentences]

def read_document_and_split_by_sentences(file_path):
    """Reads a document and splits it into sentences while keeping punctuation."""
    with open(file_path, 'r') as file:
        text = file.read().strip()

    # Preprocess text to protect abbreviations
    text = preprocess_text(text)

    # Split sentences using regex that considers punctuation
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Postprocess sentences to revert placeholders
    sentences = postprocess_sentences(sentences)

    print(f"\nOriginal Sentences from {file_path}: {sentences}")
    return sentences

def prepare_shuffled_sentences(file_path):
    """Shuffles words within sentences and adds '*' delimiter between sentences."""
    sentences = read_document_and_split_by_sentences(file_path)
    shuffled_sentences = []

    for sentence in sentences:
        # Split sentence into words
        words = sentence.split()
        # Shuffle words within the sentence
        random.shuffle(words)
        # Join the words back together after shuffling
        shuffled_sentence = ' '.join(words)
        shuffled_sentences.append(shuffled_sentence)

    # Add '*' delimiter after each sentence (except the last one)
    shuffled_text = ' * '.join(shuffled_sentences) + ' *'

    print(f"Shuffled Sentences with Delimiter for {file_path}: {shuffled_text}")

    return shuffled_text, sentences

def process_input_files():
    """Processes all files in the dataset."""
    input_folder = input("Enter the path to the folder containing subfolders A, B, and C: ")
    if not os.path.exists(input_folder):
        print("Invalid folder path. Please check and try again.")
        return None

    output_folder = os.path.join(input_folder, "Processed_Files")
    os.makedirs(output_folder, exist_ok=True)  # Create an output directory for processed files

    results = []
    for folder in ['A', 'B', 'C']:
        folder_path = os.path.join(input_folder, folder)
        if not os.path.exists(folder_path):
            print(f"Folder '{folder}' not found in the input directory. Skipping.")
            continue

        output_subfolder = os.path.join(output_folder, folder)
        os.makedirs(output_subfolder, exist_ok=True)  # Create subfolder for processed files

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if file_name.endswith(".txt"):
                shuffled_text, original_sentences = prepare_shuffled_sentences(file_path)

                # Save the processed content in the output folder with a new name
                output_file_path = os.path.join(output_subfolder, f"processed_{file_name}")
                with open(output_file_path, 'w') as file:
                    file.write(f"{shuffled_text} {'. '.join(original_sentences)}")

                results.append((output_file_path, shuffled_text, original_sentences))
                print(f"Processed file saved as: {output_file_path}")

    return results

# Uncomment to run Part-1
results = process_input_files()


Enter the path to the folder containing subfolders A, B, and C: /content/drive/MyDrive/Data (2) (1)/Data

Original Sentences from /content/drive/MyDrive/Data (2) (1)/Data/A/A-30.txt: ['I slip into the water and push off quickly before the man swimming like a breast-stroking porpoise gets any closer.', 'Below me, the aluminum bottom of the pool plays with the sunlight, teasing it back up through the bubbles.', 'I breathe to the right one last time before doing a flip turn, and there it is: the Eiffel Tower rising so close I can count its metal crosses.', 'The pool windows offer an unobstructed, third-story view.', 'Swimming in Paris is a full-on cultural experience.', 'Many public pools don’t just feel like historical monuments, they are historical monuments.', 'Backstroking beneath the buttresses stretching across the vaulted ceiling of the 99-year-old Butte-aux-Cailles pool feels like backstroking through a cathedral.', 'But after a year of swimming in Paris, it’s the smaller cultural

In [115]:
# @title
import os
import random
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import openai

# Function for getting completion from the specified model
def get_completion(prompt, model="gpt-4o", seed=420):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        seed=seed,
    )
    return response.choices[0].message.content




# Function for reconstruction using GPT models
def llm_reconstruction(shuffled_chunks, p, model):
    prompt = f"""
     You are an advanced language model tasked with reconstructing {𝑝} sentences from shuffled words. The input

consists of {p} groups of words, separated by the delimiter *. Each group corresponds to one sentence.

Your task:

Reconstruct exactly {p} sentences—no more, no less.

Use each group of words to create one sentence, ensuring that punctuation remains attached to its respective

 word. Label each sentence in your output sequentially as Sentence 1:, Sentence 2:, ..., Sentence \( p \):.

Count and verify the total number of sentences in your output. If the count does not equal {p}, revise your output until it does. Provide only the reconstructed

sentences in your output, without explanations or additional content.

Strictly adhere to the following rules:



Reconstruct exactly {p} sentences, maintaining the original order of sentences. Do not create more or fewer sentences or break any words apart.

Use each word in the provided lists exactly once within its respective sentence. Do not leave any word unused or use a word more than once.

Do not add any letters, fragments, or words that are not in the provided lists.

Keep punctuations attached to their original words. Do not move any punctuations to other words or alter them in any way.

Ensure that each reconstructed sentence is grammatically correct, meaningful, and coherent.

Provide only the reconstructed paragraph as your output, without any additional context, explanations, or modifications.

Count and verify the total number of sentences in your output. If the count does not equal {p}, revise your output until it does.

Note: The delimiter "*" is used to separate the shuffled words of each sentence. Use it to identify where each sentence begins and ends in the provided data.



 Example Input:

 world! Hello * test. is a This *  it, doing? How's *



Example Output:

Sentence 1: Hello world!

Sentence 2: This is a test.

Sentence 3: How's it doing?



Now process the following input:

{shuffled_chunks}
    """
    try:
        reconstructed_paragraph = get_completion(prompt, model=model)
        if not reconstructed_paragraph:
            raise ValueError("No response from the model.")

        # Extract sentences
        sentences = re.findall(r'^Sentence \d+:(.*)', reconstructed_paragraph, re.MULTILINE)
        reconstructed_sentences = {f"Sentence {i+1}": sentence.strip() for i, sentence in enumerate(sentences)}
        return reconstructed_sentences, len(sentences)
    except Exception as e:
        print(f"Error during LLM reconstruction: {e}")
        print(f"shuffled_chunks: {shuffled_chunks}")
        return {}, 0

# Function to prepare shuffled sentences with delimiters
def prepare_shuffled_sentences(file_path):
    with open(file_path, 'r') as file:
        text = file.read().strip()

    sentences = re.split(r'(?<=[.!?]) +', text)
    shuffled_sentences = []

    for sentence in sentences:
        words = sentence.split()
        random.shuffle(words)
        shuffled_sentence = ' '.join(words)
        shuffled_sentences.append(shuffled_sentence)

    shuffled_text = ' * '.join(shuffled_sentences) + ' *'
    return shuffled_text, sentences

# Function to compute embeddings
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Function to process a specific folder and compute results
def process_folder(base_path, folder_name):
    results = []
    folder_path = os.path.join(base_path, folder_name)

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            try:
                p = int(file_name.split("-")[1].split(".")[0])
            except (IndexError, ValueError):
                print(f"Could not extract `p` value from file name {file_name}. Skipping.")
                continue

            with open(file_path, 'r') as file:
                content = file.read()

            # Prepare shuffled text
            shuffled_text, _ = prepare_shuffled_sentences(file_path)
            print(f"Processing File {file_name}:")
            #print(f"Shuffled Text:\n{shuffled_text}")

            # Attempt Reconstruction
            reconstructed_paragraph, num_sentences = llm_reconstruction(shuffled_text, p, model="gpt-4o")
            print(f"\nReconstructed Paragraph for {file_name}:\n{reconstructed_paragraph}")
            print(f"Number of Sentences Reconstructed: {num_sentences}")

            if not reconstructed_paragraph:
                print(f"Reconstruction failed for {file_name}. Skipping similarity computation.")
                continue

            # Compute cosine similarity
            try:
                embed_original = get_embedding(content, model="text-embedding-3-large")
                embed_rebuilt = get_embedding(' '.join(reconstructed_paragraph.values()), model="text-embedding-3-large")
                similarity = cosine_similarity([embed_original], [embed_rebuilt])[0][0]

                # Append results
                results.append({
                    "File": file_name,
                    "p": p,
                    "Cosine Similarity": similarity,
                    "Reconstructed Sentences": num_sentences,
                    "Dataset": folder_name,
                })
            except Exception as embed_error:
                print(f"Error during embedding computation for {file_name}: {str(embed_error)}")

    # Calculate average and standard deviation for similarity
    df = pd.DataFrame(results)
    if not df.empty:
        avg_similarity = df["Cosine Similarity"].mean()
        std_similarity = df["Cosine Similarity"].std()
        print(f"Dataset: {folder_name} - Average Similarity: {avg_similarity}, Standard Deviation: {std_similarity}")
    else:
        avg_similarity, std_similarity = 0, 0
        print(f"Dataset: {folder_name} - No valid results.")

    return df, avg_similarity, std_similarity

# Function to calculate and create a summary table across datasets
def create_summary_table(results_by_folder):
    summary_data = {
        "Dataset": [],
        "(Avg, Std)": []
    }

    for folder_name, (df, avg_similarity, std_similarity) in results_by_folder.items():
        summary_data["Dataset"].append(folder_name)
        summary_data["(Avg, Std)"].append(f"({avg_similarity:.2f}, {std_similarity:.2f})")

    summary_df = pd.DataFrame(summary_data)
    return summary_df

# Define the base path to the dataset
base_path = "/content/drive/MyDrive/Data (2) (1)/Data/Processed_Files"

# Process each folder individually and store results
results_by_folder = {}
print("Processing folder A:")
results_by_folder["A"] = process_folder(base_path, "A")

print("\nProcessing folder B:")
results_by_folder["B"] = process_folder(base_path, "B")

print("\nProcessing folder C:")
results_by_folder["C"] = process_folder(base_path, "C")

# Generate the summary table
print("\nGenerating summary table...")
summary_table = create_summary_table(results_by_folder)

# Display the summary table
print("\nSummary Table:")
print(summary_table)


Processing folder A:
Processing File processed_A-30.txt:

Reconstructed Paragraph for processed_A-30.txt:
{'Sentence 1': 'Words: Shuffled feel like my last beauty monuments.', 'Sentence 2': 'Piscine Amiraux, the 99-year-old Art Deco pool, has been used mostly during the coronavirus pandemic.', 'Sentence 3': 'The historical city’s swimming pools offer a guide, living with unobstructed views of the Eiffel Tower.', 'Sentence 4': 'The city’s cultural style opens up to the historical monuments and many swimming pools.', 'Sentence 5': 'I didn’t feel like swimming, but the pool quickly introduced me to a new experience.', 'Sentence 6': 'The swimming pool has a third-story dome with a skylight that opens up to the city.', 'Sentence 7': 'The city’s swimming pools are mostly used by the public, and the rules are strict.', 'Sentence 8': 'The pool’s changing rooms are separate for men and women, with showers in each.', 'Sentence 9': 'The pool’s gender rules are strict, but the showers are coed.', 

EXPERIMENT-3

In [ ]:
# @title
import os
import random
import re

# Common abbreviations to exclude from sentence splitting
ABBREVIATIONS = {"Mr.", "Mrs.", "Dr.", "Ms.", "Jr.", "Sr.", "St.", "Prof.", "Inc.", "Ltd.", "Co."}

def preprocess_text(text):
    """Preprocesses the text to protect abbreviations from being split."""
    for abbrev in ABBREVIATIONS:
        # Replace abbreviations with a placeholder to prevent splitting
        text = text.replace(abbrev, abbrev.replace(".", "<DOT>"))
    return text

def postprocess_sentences(sentences):
    """Reverts placeholders in sentences back to their original abbreviations."""
    return [sentence.replace("<DOT>", ".") for sentence in sentences]

def read_document_and_split_by_sentences(file_path):
    """Reads a document and splits it into sentences while keeping punctuation."""
    with open(file_path, 'r') as file:
        text = file.read().strip()

    # Preprocess text to handle abbreviations
    text = preprocess_text(text)

    # Split sentences using regex that considers punctuation (period, exclamation mark, question mark).
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Postprocess sentences to restore abbreviations
    sentences = postprocess_sentences(sentences)

    print(f"\nOriginal Sentences from {file_path}: {sentences}")
    return sentences

def prepare_shuffled_sentences_and_words(file_path):
    """Shuffles entire sentences, then shuffles words within sentences, and adds '*' delimiter."""
    # Read and split the document into sentences
    sentences = read_document_and_split_by_sentences(file_path)

    # Shuffle the sentences
    random.shuffle(sentences)
    print(f"Shuffled Sentences for {file_path}: {sentences}")

    # Shuffle words within each sentence
    shuffled_sentences_with_words = []
    for sentence in sentences:
        words = sentence.split()  # Split sentence into words
        random.shuffle(words)  # Shuffle words
        shuffled_sentence = ' '.join(words)  # Recombine words into a sentence
        shuffled_sentences_with_words.append(shuffled_sentence)

    # Add '*' delimiter between shuffled sentences
    shuffled_text = ' * '.join(shuffled_sentences_with_words)

    print(f"Shuffled Words in Sentences with Delimiter for {file_path}: {shuffled_text}")

    return shuffled_text, sentences

def process_input_files():
    """Processes all files in the dataset."""
    input_folder = input("Enter the path to the folder containing subfolders A, B, and C: ")
    if not os.path.exists(input_folder):
        print("Invalid folder path. Please check and try again.")
        return None

    output_folder = os.path.join(input_folder, "Processed_Files")
    os.makedirs(output_folder, exist_ok=True)  # Create an output directory for processed files

    results = []
    for folder in ['A', 'B', 'C']:
        folder_path = os.path.join(input_folder, folder)
        if not os.path.exists(folder_path):
            print(f"Folder '{folder}' not found in the input directory. Skipping.")
            continue

        output_subfolder = os.path.join(output_folder, folder)
        os.makedirs(output_subfolder, exist_ok=True)  # Create subfolder for processed files

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if file_name.endswith(".txt"):
                shuffled_text, original_sentences = prepare_shuffled_sentences_and_words(file_path)

                # Save the processed content in the output folder with a new name
                output_file_path = os.path.join(output_subfolder, f"processed_{file_name}")
                with open(output_file_path, 'w') as file:
                    # Save shuffled text with delimiters and original sentences
                    file.write(f"{shuffled_text}\nOriginal Sentences: {' '.join(original_sentences)}")

                results.append((output_file_path, shuffled_text, original_sentences))
                print(f"Processed file saved as: {output_file_path}")

    return results

# Uncomment to run Part-1
results = process_input_files()


Enter the path to the folder containing subfolders A, B, and C: /content/drive/MyDrive/Data (2) (1)/Data

Original Sentences from /content/drive/MyDrive/Data (2) (1)/Data/A/A-30.txt: ['I slip into the water and push off quickly before the man swimming like a breast-stroking porpoise gets any closer.', 'Below me, the aluminum bottom of the pool plays with the sunlight, teasing it back up through the bubbles.', 'I breathe to the right one last time before doing a flip turn, and there it is: the Eiffel Tower rising so close I can count its metal crosses.', 'The pool windows offer an unobstructed, third-story view.', 'Swimming in Paris is a full-on cultural experience.', 'Many public pools don’t just feel like historical monuments, they are historical monuments.', 'Backstroking beneath the buttresses stretching across the vaulted ceiling of the 99-year-old Butte-aux-Cailles pool feels like backstroking through a cathedral.', 'But after a year of swimming in Paris, it’s the smaller cultural

In [113]:
# @title
import os
import random
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import openai

# Function for getting completion from the specified model
def get_completion(prompt, model="gpt-4o", seed=420):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        seed=seed,
    )
    return response.choices[0].message.content




# Function for reconstruction using GPT models
def llm_reconstruction(shuffled_chunks, p, model):
    prompt = f"""
You are an advanced language model.Reconstruct exactly {p} sentences from the following shuffled groups of words. Each group is separated by "*".
- Use all words in a group exactly once to form one complete and grammatically correct sentence.
- Do not use words from one group in another.
- Maintain original punctuation attached to words.

Label each reconstructed sentence as:
Sentence 1: <reconstructed sentence>
Sentence 2: <reconstructed sentence>
...
Sentence {p}: <reconstructed sentence>

Input: {shuffled_chunks}
    """
    try:
        reconstructed_paragraph = get_completion(prompt, model=model)
        if not reconstructed_paragraph:
            raise ValueError("No response from the model.")

        # Extract sentences
        sentences = re.findall(r'^Sentence \d+:(.*)', reconstructed_paragraph, re.MULTILINE)
        reconstructed_sentences = {f"Sentence {i+1}": sentence.strip() for i, sentence in enumerate(sentences)}
        return reconstructed_sentences, len(sentences)
    except Exception as e:
        print(f"Error during LLM reconstruction: {e}")
        print(f"shuffled_chunks: {shuffled_chunks}")
        return {}, 0

# Function to prepare shuffled sentences with delimiters
def prepare_shuffled_sentences(file_path):
    with open(file_path, 'r') as file:
        text = file.read().strip()

    sentences = re.split(r'(?<=[.!?]) +', text)
    shuffled_sentences = []

    for sentence in sentences:
        words = sentence.split()
        random.shuffle(words)
        shuffled_sentence = ' '.join(words)
        shuffled_sentences.append(shuffled_sentence)

    shuffled_text = ' * '.join(shuffled_sentences) + ' *'
    return shuffled_text, sentences

# Function to compute embeddings
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# Function to process a specific folder and compute results
def process_folder(base_path, folder_name):
    results = []
    folder_path = os.path.join(base_path, folder_name)

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            try:
                p = int(file_name.split("-")[1].split(".")[0])
            except (IndexError, ValueError):
                print(f"Could not extract `p` value from file name {file_name}. Skipping.")
                continue

            with open(file_path, 'r') as file:
                content = file.read()

            # Prepare shuffled text
            shuffled_text, _ = prepare_shuffled_sentences(file_path)
            print(f"Processing File {file_name}:")
            #print(f"Shuffled Text:\n{shuffled_text}")

            # Attempt Reconstruction
            reconstructed_paragraph, num_sentences = llm_reconstruction(shuffled_text, p, model="gpt-4o")
            print(f"\nReconstructed Paragraph for {file_name}:\n{reconstructed_paragraph}")
            print(f"Number of Sentences Reconstructed: {num_sentences}")

            if not reconstructed_paragraph:
                print(f"Reconstruction failed for {file_name}. Skipping similarity computation.")
                continue

            # Compute cosine similarity
            try:
                embed_original = get_embedding(content, model="text-embedding-3-large")
                embed_rebuilt = get_embedding(' '.join(reconstructed_paragraph.values()), model="text-embedding-3-large")
                similarity = cosine_similarity([embed_original], [embed_rebuilt])[0][0]

                # Append results
                results.append({
                    "File": file_name,
                    "p": p,
                    "Cosine Similarity": similarity,
                    "Reconstructed Sentences": num_sentences,
                    "Dataset": folder_name,
                })
            except Exception as embed_error:
                print(f"Error during embedding computation for {file_name}: {str(embed_error)}")

    # Calculate average and standard deviation for similarity
    df = pd.DataFrame(results)
    if not df.empty:
        avg_similarity = df["Cosine Similarity"].mean()
        std_similarity = df["Cosine Similarity"].std()
        print(f"Dataset: {folder_name} - Average Similarity: {avg_similarity}, Standard Deviation: {std_similarity}")
    else:
        avg_similarity, std_similarity = 0, 0
        print(f"Dataset: {folder_name} - No valid results.")

    return df, avg_similarity, std_similarity

# Function to calculate and create a summary table across datasets
def create_summary_table(results_by_folder):
    summary_data = {
        "Dataset": [],
        "(Avg, Std)": []
    }

    for folder_name, (df, avg_similarity, std_similarity) in results_by_folder.items():
        summary_data["Dataset"].append(folder_name)
        summary_data["(Avg, Std)"].append(f"({avg_similarity:.2f}, {std_similarity:.2f})")

    summary_df = pd.DataFrame(summary_data)
    return summary_df

# Define the base path to the dataset
base_path = "/content/drive/MyDrive/Data (2) (1)/Data/Processed_Files"

# Process each folder individually and store results
results_by_folder = {}
print("Processing folder A:")
results_by_folder["A"] = process_folder(base_path, "A")

print("\nProcessing folder B:")
results_by_folder["B"] = process_folder(base_path, "B")

print("\nProcessing folder C:")
results_by_folder["C"] = process_folder(base_path, "C")

# Generate the summary table
print("\nGenerating summary table...")
summary_table = create_summary_table(results_by_folder)

# Display the summary table
print("\nSummary Table:")
print(summary_table)


Processing folder A:
Processing File processed_A-30.txt:

Reconstructed Paragraph for processed_A-30.txt:
{'Sentence 1': 'My last feel of the pandemic ceiling beauty seem unobstructed, like monuments.', 'Sentence 2': 'The city’s historical pools are mostly open during the coronavirus pandemic.', 'Sentence 3': 'But the changing room, separate and done, moved one to breathe it.', 'Sentence 4': 'It’s stretching right in the edge of the pool.', 'Sentence 5': 'By 2006, when no rooms were doing.', 'Sentence 6': 'Most are a lifeguard, social problems visited across the city.', 'Sentence 7': 'I gleaned with a man who renovates the modern tiles.', 'Sentence 8': 'I find my rising to the side of the lake.', 'Sentence 9': 'Schneck, the writer, swam across the pool.', 'Sentence 10': 'The view of the city is: unobstructed, like a third-story pool.', 'Sentence 11': 'The pool comes with a skylight and aluminum windows.', 'Sentence 12': 'The pool is a full-on cultural experience in Paris.', 'Sentence 1